In [1]:
import os
import json

import pandas as pd
import numpy as np

In [2]:
file_name = 'result_yolov5l6_pretrained_highaug_imgsize1280_epoch120_wbf_ensemble_HONEY.csv'
pseudo = pd.read_csv(file_name)
pseudo

,PredictionString,image_id
0,7 0.9817209243774414 602.0529174804688 519.373...,test/0000.jpg
1,5 0.9219956994056702 343.978759765625 252.1072...,test/0001.jpg
2,1 0.9528318047523499 74.50826263427734 272.086...,test/0002.jpg
3,9 0.9029123783111572 143.1494140625 256.653839...,test/0003.jpg
4,0 0.715968132019043 193.74159240722656 252.522...,test/0004.jpg
...,...,...
4866,1 0.5722888469696045 291.8671875 348.466918945...,test/4866.jpg
4867,3 0.9197988510131836 415.5954284667969 316.431...,test/4867.jpg
4868,7 0.9602166414260864 115.62899780273438 450.45...,test/4868.jpg
4869,0 0.675341784954071 0.8300445675849915 240.166...,test/4869.jpg


In [3]:
data_dir = '/opt/ml/dataset'
test_json = json.load(open(os.path.join(data_dir,'test.json')))

In [4]:
thr = 0.4

for i in range(4871):

    if type(pseudo['PredictionString'][i])==str: # pred가 없는 이미지는 무시
        for j in range(len(pseudo['PredictionString'][i].split(' '))//6):
            o = pseudo['PredictionString'][i].split(' ')
            id, score, xmin, ymin, xmax, ymax = map(float,o[6*j: 6*(j+1)])
            if score<thr:
                continue
            id, xmin, ymin, xmax, ymax = map(int, [id, xmin, ymin, xmax, ymax])
            w = xmax - xmin
            h = ymax - ymin

            info = dict()
            info['image_id'] = i
            info['category_id'] = id
            info['area'] = w*h
            info['bbox'] = [xmin, ymin, w, h]
            info['iscrowd'] = 0 # 겹친 박스가 있는지 여부
            info['id'] = j # 박스 id
            test_json['annotations'].append(info)
            

In [6]:
test_json['annotations']

[{'image_id': 0,
  'category_id': 7,
  'area': 178416,
  'bbox': [602, 519, 354, 504],
  'iscrowd': 0,
  'id': 0},
 {'image_id': 0,
  'category_id': 7,
  'area': 100380,
  'bbox': [216, 51, 239, 420],
  'iscrowd': 0,
  'id': 1},
 {'image_id': 0,
  'category_id': 7,
  'area': 62196,
  'bbox': [118, 419, 213, 292],
  'iscrowd': 0,
  'id': 2},
 {'image_id': 0,
  'category_id': 7,
  'area': 54600,
  'bbox': [447, 601, 200, 273],
  'iscrowd': 0,
  'id': 3},
 {'image_id': 0,
  'category_id': 7,
  'area': 61020,
  'bbox': [652, 335, 226, 270],
  'iscrowd': 0,
  'id': 4},
 {'image_id': 0,
  'category_id': 7,
  'area': 83647,
  'bbox': [391, 186, 233, 359],
  'iscrowd': 0,
  'id': 5},
 {'image_id': 0,
  'category_id': 5,
  'area': 4399,
  'bbox': [342, 641, 53, 83],
  'iscrowd': 0,
  'id': 6},
 {'image_id': 0,
  'category_id': 5,
  'area': 3886,
  'bbox': [244, 681, 58, 67],
  'iscrowd': 0,
  'id': 7},
 {'image_id': 0,
  'category_id': 1,
  'area': 51480,
  'bbox': [567, 103, 198, 260],
  'iscr

In [5]:
with open(os.path.join(data_dir, f"pseudo_{file_name.split('.')[0]}.json"),'w') as f:
        json.dump(test_json, f, indent=4)